In [2]:
from Bio import AlignIO

from glob import glob

In [6]:
# Found this on youtube

for filename in glob("*.fasta"):
    with open(filename) as f:
        output = str(filename)
        output += "-aligned.fasta"
        in_file = str(filename)
        from Bio.Align.Applications import MafftCommandline
        mafft_cline = MafftCommandline(input= in_file)
        print(mafft_cline)
        stdout, stderr = mafft_cline()
        with open(output, "w") as handle:
            handle.write(stdout)

mafft mollusks_hemocyanin.fasta


In [7]:
%%bash

ls

mollusks_hemocyanin.fasta
mollusks_hemocyanin.fasta-aligned.fasta
Untitled.ipynb


In [27]:
align = AlignIO.parse("mollusks_hemocyanin.fasta-aligned.fasta", "fasta")
for i in align:
    print(i)

SingleLetterAlphabet() alignment with 194 rows and 5618 columns
----------------------------------LVQF------...--- CAC20588.1
---------------------------MALLWSFLA--------...--- QCX35557.1
--------------------------------------------...--- ADT91166.1
--------------------------------------------...--- ADT91165.1
--------------------------------------------...--- ADT91164.1
--------------------------------------------...--- ADT91163.1
--------------------------------------------...--- AAQ03083.1
-------------MRHLNHLPLLYLTVILTFWMTGL--------...--- AGV74427.1
--------------------------------------------...--- AAA29390.1
--------------------------------------------...--- CBW46988.1
--------------------------------------------...--- CAG47101.2
--------------------------------------------...--- CAG47102.1
--------------------------------------------...--- CAG47100.1
--------------------------------------------...--- ACF70600.1
--------------------------------------------...--- ADT91171.1
------

In [26]:
align = AlignIO.read("mollusks_hemocyanin.fasta-aligned.fasta", "fasta")
for i in align:
    print(i)

ID: CAC20588.1
Name: CAC20588.1
Description: CAC20588.1 hemocyanin, partial [Haliotis tuberculata]
Number of features: 0
Seq('----------------------------------LVQF----------------...---', SingleLetterAlphabet())
ID: QCX35557.1
Name: QCX35557.1
Description: QCX35557.1 Hemocyanin [Lymnaea stagnalis]
Number of features: 0
Seq('---------------------------MALLWSFLA------------------...---', SingleLetterAlphabet())
ID: ADT91166.1
Name: ADT91166.1
Description: ADT91166.1 hemocyanin, partial [Spirula spirula]
Number of features: 0
Seq('------------------------------------------------------...---', SingleLetterAlphabet())
ID: ADT91165.1
Name: ADT91165.1
Description: ADT91165.1 hemocyanin, partial [Spirula spirula]
Number of features: 0
Seq('------------------------------------------------------...---', SingleLetterAlphabet())
ID: ADT91164.1
Name: ADT91164.1
Description: ADT91164.1 hemocyanin, partial [Spirula spirula]
Number of features: 0
Seq('-------------------------------------------------

In [17]:
len(align[110])

5618

In [48]:
from Bio import SeqIO
with open("mollusks_hemocyanin.fasta-aligned.fasta") as handle:
    for record in SeqIO.parse(handle, "fasta") :
        print(record.description)
        print(record.seq)

CAC20588.1 hemocyanin, partial [Haliotis tuberculata]
----------------------------------LVQF----------------------LLVALVVGAGADNVVRKDVSHLTVDEVQALHGALHDVTAS-TGPLSFEDITSYHAAPAS-C-DYKGRKIACCVHGMPSFPFWHRAYVVQAERALLSKRK-TVGMPYWDWTQTLTHLPSLVTEPIYIDSKGGKAQTNYWYRGEIAF------INKK--TARAVDDRLFEKVEPGHYTHLMETVLDALEQDEFCKFEIQFELAHNAIHYLVGGKFEYSMSNLEYTSYDPIFFLHHSNVDRLFAIWQRLQELRGKNPN------AMDCAHELAHQQLQPFNRDSNPVQLTKDHSTPADLFDYK--QLGYSYDSLN-LNGMTPEQLKTELDERHSKERAFASFRLSGFGGSANVVVYACVP----------DDD----PRSDDYCEK-AGDFFILGGQSEMPWRFYRPFFYDVTEAVHHLGVPLSGHYYVKTELFSVNGTALSPDLLPQPTVAYRPGKGHLDPPV-----------HHRHDDDLIVRKNIDHLTREEEYELRMALERFQADTSVDGYQATVEYHGLPARC--PRPDAKVRFACCMHGMASFPHWHRLFVTQVEDALVRRGSPIGV-----PYWDWTKPMTHLPDLASNETYVDPYGH--THHNPFFNANISFEEGHHHTSRMI-DSKLFAPVAFGEHSHLFDGILYAFEQEDFC------------DFEIQFELVHNSIHAWIGGSEDYSMATLHYTAFDPIFYLHHSNVDRLWAIWQALQIRRHKPYQAHCAQSVEQLPMKPFAFPSPLNNNEKTHSHSVPTDIYDYEEVLHYSYDDLTFGGMNLEEIEEAIHLR-QQHERVFAGFLLAGIGTSALVDIFINK-PGNQ-PLKAGDIAILGGAKEMPWAFDRLYKVEITDSLKTLSLDVDGDYEVTFKIH

In [40]:
from Bio import SeqIO
record = SeqIO.read("mollusks_hemocyanin.fasta", "fasta")
record

ValueError: More than one record found in handle